**Notebook de prétraitement des données textuelles : correction et ponctuation**  

Ce notebbok utiise le modèle Mixtral 8B pour effectuer une correction et une ponctuation des textes du corpus.

Ce notebbok a été construit dans colab, il est conseillé de le faire tourner aussi dans Colab.

On commence par les importations :

In [2]:
!pip install -qU langchain_mistralai
!pip install -qU langchain_community
from langchain_community.document_loaders import WebBaseLoader

In [3]:
import pandas as pd

On mentionne les clefs API, à aller chercher sur le site de mistral et sur le site de langsmith :

In [4]:
import os

os.environ['MISTRAL_API_KEY']= ""
os.environ['LANGSMITH_API_KEY']=""

On charge le modèle, et on rédige le prompt. Il est important de forcer le modèle à ne pas rajouter de texte en plus, sinon, il commente le texte.

In [5]:
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain_mistralai import ChatMistralAI

# Initialiser le modèle (remplace "temperature" et "api_key" par tes paramètres)
llm = ChatMistralAI(
    model="open-mixtral-8x22b",
    temperature=0,
    max_retries=2,
    # other params...
)

# Définir le template du prompt
prompt = PromptTemplate(
    input_variables=["text"],
    template=(

    """Tu es un assistant spécialisé en correction de texte. Voici un texte sur le changement climatique :

    {text}

    Corrige les fautes d'orthographe, corrige les noms propres s'ils contiennent des fautes, et ajoute de la ponctuation si nécessaire.
    Tu dois corriger le texte uniquement si cela te paraît nécessaire et que tu es sûr de toi. **Ne modifie pas le contenu ou les idées du texte.**

    Retourne uniquement le texte corrigé, sans aucun autre message ou explication."""
    )

        )

# ✅ Définir la chaîne ici
chain = LLMChain(prompt=prompt, llm=llm)

/tmp/ipython-input-5-1630892426.py:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm)


Je veux appliquer mon prompt à un ensemble de fichiers zippés en un seul fichier. Le code suivant dézippe le fichier, et créé un objet documents qui est un dictionnaire. Cet objet documents contient deux clef, titre et contenu.

In [13]:
import os
import zipfile
import shutil  # pour supprimer un dossier

zip_path = "/content/dernier_dossier.zip"
extract_dir = "extracted_files"

# Supprimer le dossier s'il existe déjà pour éviter de mélanger les fichiers
if os.path.exists(extract_dir):
    shutil.rmtree(extract_dir)

# Initialiser l'objet "documents"
docs = []

# Extraire le nouveau ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Parcourir les fichiers texte extraits
for filename in os.listdir(extract_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(extract_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        docs.append({
            "file_name": filename,
            "content": content
        })

# Afficher un aperçu
print(docs)


[{'file_name': 'thinkerview_jean_marc_jancovict.txt', 'content': "[Musique] Jean-Marc Jancovici bonsoir bonsoir nous vous retrouvons pour une chaîne internet qui s'appelle thinkerview nous sommes en direct est-ce que vous pouvez vous présenter succinctement je à part très né dans ce studio je travaille dans une société qui s'appelle carbone cap qui essaye de confronter les activités économiques à la contrainte énergie et climat je suis président d'une association qui s'appelle Dechy projet qui est le pendant militant on va dire de la même chose je suis à mes heures perdues enseignants et à mes heures encore plus perdu j'écris de temps en temps quelque chose qui essaye de compenser le cap sur 20 que j'ai magistralement reçu à l'écrit du bac de français il y a très longtemps félicitations donc monsieur Jancovici Jean-Marc on a décidé de vous faire venir aujourd'hui pour parler sobriété des carbonations peut-être un peu mais surtout la différence entre une sobriété voulue et une sobriété 

On créé un objet dataframe, pour itérer facilement sur notre corpus :

In [14]:
mon_df = pd.DataFrame(docs)

5

La boucle suivante applique le prompt à l'objet document. Pour éviter de surcharger l'api, on ajoute une pause de deux secondes entre chaque fichier. Aussi, les textes sont découpés en extraits de 3000 caractères, sinon le LLM est surchargé.

Il faudra compter plusieurs heures pour plusieurs gros fichiers.

In [10]:
import time
import textwrap
from textwrap import wrap
from tqdm import tqdm  # barre de progression

In [16]:
results = []

# Boucle avec barre de progression
for i, row in tqdm(mon_df.iterrows(), total=len(mon_df), desc="Correction des textes"):
    content = row['content']
    file_name = row['file_name']
    corrected_chunks = []

    try:
        chunks = textwrap.wrap(content, 3000)  # découpe le texte

        for chunk in chunks:
            response = chain.invoke({"text": chunk})
            # Extraire le texte corrigé si c’est un dict
            corrected_text_part = response['text'] if isinstance(response, dict) else response
            corrected_chunks.append(corrected_text_part)
            time.sleep(1.5)  # pause pour éviter la surcharge de l’API

        corrected_text = " ".join(corrected_chunks)

    except Exception as e:
        print(f"Erreur pour {file_name} : {e}")
        corrected_text = None

    results.append(corrected_text)

# Ajouter les résultats dans une nouvelle colonne
mon_df['corrected_content'] = results


Correction des textes: 100%|██████████| 5/5 [52:45<00:00, 633.03s/it]


On stocke l'objet result dans un csv pour sauvegarder les résultats.

In [18]:
import pandas as pd
df = pd.DataFrame(results)

df.to_csv("csv_corpus_temoin_corrige_4.csv", index=False)